In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('../data/raw/data.csv')

In [3]:
df.head()

,Unnamed: 0,model_name,model_year,kms_driven,owner,location,price
0,0,Yamaha Fazer Dlx Standard,2014 model,"12,600 kms",1st Owner,Raigad,"34,999"
1,1,Yamaha RX135 Standard,1996 model,"14,500 kms",2nd Owner,Madurai,"85,000"
2,2,Bajaj Pulsar 150 [2001-2011] Kick Start,2007 model,"51,000 kms",2nd Owner,Bangalore,"25,000"
3,3,Royal Enfield Electra Twinspark Standard,2013 model,"57,500 kms",1st Owner,Bangalore,"1,00,000"
4,4,Hero Honda Glamour Alloy Drum Self,2006 model,"69,569 kms",2nd Owner,Azamgarh,"21,000"


In [4]:
df.shape

(1327, 7)

In [5]:
df = df.drop(columns=['Unnamed: 0'])

In [6]:
df.head()

,model_name,model_year,kms_driven,owner,location,price
0,Yamaha Fazer Dlx Standard,2014 model,"12,600 kms",1st Owner,Raigad,"34,999"
1,Yamaha RX135 Standard,1996 model,"14,500 kms",2nd Owner,Madurai,"85,000"
2,Bajaj Pulsar 150 [2001-2011] Kick Start,2007 model,"51,000 kms",2nd Owner,Bangalore,"25,000"
3,Royal Enfield Electra Twinspark Standard,2013 model,"57,500 kms",1st Owner,Bangalore,"1,00,000"
4,Hero Honda Glamour Alloy Drum Self,2006 model,"69,569 kms",2nd Owner,Azamgarh,"21,000"


## Data Cleaning

In [7]:
# Remove commas from kms_driven and price columns
df['kms_driven'] = df['kms_driven'].str.replace(',','')
df['kms_driven'] = df['kms_driven'].str.replace('kms','')
df['price'] = df['price'].str.replace(',','')

In [8]:
df.head()

,model_name,model_year,kms_driven,owner,location,price
0,Yamaha Fazer Dlx Standard,2014 model,12600,1st Owner,Raigad,34999
1,Yamaha RX135 Standard,1996 model,14500,2nd Owner,Madurai,85000
2,Bajaj Pulsar 150 [2001-2011] Kick Start,2007 model,51000,2nd Owner,Bangalore,25000
3,Royal Enfield Electra Twinspark Standard,2013 model,57500,1st Owner,Bangalore,100000
4,Hero Honda Glamour Alloy Drum Self,2006 model,69569,2nd Owner,Azamgarh,21000


In [9]:
# Let's make the model_year columm as number only by removing the 'model' string
df['model_year'] = df['model_year'].str.replace('model','')

In [10]:
df.head()

,model_name,model_year,kms_driven,owner,location,price
0,Yamaha Fazer Dlx Standard,2014,12600,1st Owner,Raigad,34999
1,Yamaha RX135 Standard,1996,14500,2nd Owner,Madurai,85000
2,Bajaj Pulsar 150 [2001-2011] Kick Start,2007,51000,2nd Owner,Bangalore,25000
3,Royal Enfield Electra Twinspark Standard,2013,57500,1st Owner,Bangalore,100000
4,Hero Honda Glamour Alloy Drum Self,2006,69569,2nd Owner,Azamgarh,21000


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1327 entries, 0 to 1326
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   model_name  1327 non-null   object
 1   model_year  1327 non-null   object
 2   kms_driven  1327 non-null   object
 3   owner       1327 non-null   object
 4   location    1327 non-null   object
 5   price       1327 non-null   object
dtypes: object(6)
memory usage: 62.3+ KB


In [12]:
df['model_year'] = pd.to_numeric(df['model_year'],errors='coerce')
df['kms_driven'] = pd.to_numeric(df['kms_driven'],errors='coerce')
df['price'] = pd.to_numeric(df['price'],errors='coerce')

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1327 entries, 0 to 1326
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   model_name  1327 non-null   object
 1   model_year  1327 non-null   int64 
 2   kms_driven  1327 non-null   int64 
 3   owner       1327 non-null   object
 4   location    1327 non-null   object
 5   price       1327 non-null   int64 
dtypes: int64(3), object(3)
memory usage: 62.3+ KB


In [16]:
df.isnull().sum()

model_name    0
model_year    0
kms_driven    0
owner         0
location      0
price         0
dtype: int64

In [14]:
df.to_csv('../data/processed/data.csv',index=False)